%md

# Transformação para Silver – Análises Agregadas de Trades DEX

Este notebook constrói a **camada Silver** do pipeline de dados Ethereum, com foco em análises agregadas e estruturadas para facilitar a exploração e visualização das transações de exchanges descentralizadas (DEX). A base utilizada é a tabela `workspace.ethereum.fato_dex_trades`, já enriquecida com chaves de dimensões e estruturada na camada Bronze.

Pelo escopo deste projeto, a camada Silver será também a camada analítica final, não sendo necessária a construção de uma camada Gold. Todas as visões geradas nesta etapa já são otimizadas para consumo direto por dashboards, notebooks exploratórios e ferramentas de visualização. As agregações, filtros e joins com dimensões foram aplicados de forma que as tabelas resultantes entreguem valor analítico imediato, reduzindo a complexidade e o custo de manutenção de uma camada adicional.

---

## Objetivo

Construir tabelas analíticas e agregadas na camada **Silver**, com os seguintes propósitos:

- Consolidar dados transacionais em visões de alto nível  
- Permitir análises temporais e comparativas por token, exchange e rede  
- Servir de base para dashboards e relatórios no Streamlit ou Power BI  
- Reduzir a complexidade de joins para os consumidores finais dos dados

---

## Etapas do Pipeline de Transformação

### 1. Criação de Visões Agregadas

A camada Silver gera **quatro visões analíticas principais**, conforme descrito abaixo:

---

### 🔹 `transacoes_token_dia_direcao_silver`

Agrupa os dados por data, nome do token e direção da operação, trazendo métricas como:

- Volume total em USD  
- Preço médio do token  
- Total de negociações (trades)  
- Quantidade média negociada por transação  

**Chaves:** `data`, `nome_dia`, `nome_token`, `moeda`, `direcao`  
**Filtro aplicado:** `quantia_trocada_usd > 10.000`

---

### 🔹 `transacoes_exchange_dia_direcao_silver`

Traz a movimentação por exchange, data e direção, agregando:

- Total de transações registradas  
- Volume em USD  
- Preço médio  
- Total de trades  

**Chaves:** `data`, `nome_dia`, `nome_exchange`, `direcao`  
**Filtro aplicado:** `quantia_trocada_usd > 5.000`

---

### 🔹 `transacoes_token_direcao_silver`

Consolida os dados por token e direção (Buy/Sell), útil para análises comparativas de comportamento de mercado:

- Total de transações  
- Volume em USD  
- Preço médio  

**Chaves:** `nome_token`, `moeda`, `direcao`  
**Filtro aplicado:** `quantia_trocada_usd > 5.000`

---

### 🔹 `analise_rede_dia_silver`

Agrega os dados por **rede blockchain**, data e dia da semana, fornecendo:

- Volume total em USD  
- Total de trades realizados  

**Chaves:** `fk_rede`, `data`, `nome_dia`  
**Filtro aplicado:** `quantia_trocada_usd > 5.000`

---

## Observações Técnicas

- Todas as tabelas foram salvas no formato **Delta Lake** com o modo `overwrite` para facilitar atualizações completas.
- Filtros de corte mínimo (`quantia_trocada_usd`) foram aplicados para eliminar ruído e melhorar a qualidade das análises.
- As dimensões utilizadas (`dim_calendario`, `dim_token`, `dim_exchange`) asseguram maior legibilidade sem comprometer a normalização.
- A camada Silver serve como **base de dados intermediária**, ideal para consumo direto por dashboards e notebooks exploratórios.

---

## Próximos Passos

- Construção de visualizações no **Streamlit** utilizando as tabelas Silver  
- Criação de uma camada **Gold**, caso sejam necessárias análises combinadas ou enriquecidas com indicadores externos  
- Monitoramento da qualidade das métricas com indicadores de consistência (ex: variações bruscas no volume ou número de trades)


# Tabela de operações por token e direção

In [0]:
from pyspark.sql.functions import col, sum, avg

(
    spark.table("workspace.ethereum.fato_dex_trades").alias("f")
    .join(spark.table("workspace.ethereum.dim_calendario").alias("c"), col("f.fk_data") == col("c.id_data"))
    .join(spark.table("workspace.ethereum.dim_token").alias("t"), col("f.fk_token_base") == col("t.id_token"))
    .filter(col("f.quantia_trocada_usd") > 10000)
    .groupBy(
        col("c.data"),
        col("c.nome_dia"),
        col("t.nome_token"),
        col("t.moeda"),
        col("f.direcao")
    )
    .agg(
        sum("f.quantia_trocada_usd").alias("volume_total_usd"),
        avg("f.preco_medio").alias("preco_medio_usd"),
        sum("f.num_trades").alias("total_trades"),
        avg("f.quantia_base").alias("media_quantia_token")
    )
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.ethereum.transacoes_token_dia_direcao_silver")
)




# Transações por Exchange e Direção
## Objetivo: Analisar volume e número de trades agrupados por exchange e direção (BUY/SELL).

In [0]:
from pyspark.sql.functions import col, count, sum, avg

(
    spark.table("workspace.ethereum.fato_dex_trades").alias("f")
    .join(
        spark.table("workspace.ethereum.dim_calendario").alias("c"),
        col("f.fk_data") == col("c.id_data")
    )
    .join(
        spark.table("workspace.ethereum.dim_exchange").alias("e"),
        col("f.fk_exchange") == col("e.id_exchange")
    )
    .filter(col("f.quantia_trocada_usd") > 5000)
    .groupBy(
        col("c.data"),
        col("c.nome_dia").alias("dia_semana"),
        col("e.nome_exchange"),
        col("f.direcao")
    )
    .agg(
        count("*").alias("total_transacoes"),
        sum("f.quantia_trocada_usd").alias("volume_usd"),
        avg("f.preco_medio").alias("preco_medio_usd"),
        sum("f.num_trades").alias("total_trades")
    )
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.ethereum.transacoes_exchange_dia_direcao_silver")
)



# Volume Agregado por Token e Direção (sem data)
## Objetivo: Visão macro de quais tokens estão sendo mais comprados ou vendidos, independentemente do dia.

In [0]:
from pyspark.sql.functions import col, count, sum, avg

(
    spark.table("workspace.ethereum.fato_dex_trades").alias("f")
    .join(
        spark.table("workspace.ethereum.dim_token").alias("t"),
        col("f.fk_token_base") == col("t.id_token")
    )
    .filter(col("f.quantia_trocada_usd") > 5000)
    .groupBy(
        col("t.nome_token"),
        col("t.moeda"),
        col("f.direcao")
    )
    .agg(
        count("*").alias("total_transacoes"),
        sum("f.quantia_trocada_usd").alias("volume_usd"),
        avg("f.preco_medio").alias("preco_medio_usd")
    )
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.ethereum.transacoes_token_direcao_silver")
)



# tabela de transações por network

In [0]:
from pyspark.sql.functions import col, sum

(
    spark.table("workspace.ethereum.fato_dex_trades").alias("f")
    .join(
        spark.table("workspace.ethereum.dim_calendario").alias("c"),
        col("f.fk_data") == col("c.id_data")
    )
    .join(
        spark.table("workspace.ethereum.dim_rede").alias("r"),
        col("f.fk_rede") == col("r.id_rede")
    )
    .filter(col("f.quantia_trocada_usd") > 5000)
    .groupBy(
        col("f.fk_rede"),
        col("r.rede"),
        col("c.data"),
        col("c.nome_dia")
    )
    .agg(
        sum("f.quantia_trocada_usd").alias("volume_total_usd"),
        sum("f.num_trades").alias("total_trades")
    )
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.ethereum.analise_rede_dia_silver")
)

